In [7]:
#Dependency
from Bio import Entrez
import csv
import xml.etree.ElementTree as ET

In [13]:
def extract(in_string,pattern):
    found=in_string.find(pattern)
    if found>=0:
        return in_string[len(pattern)+found:].split(" ")[0].strip('""')
    return ""

def getAccession(id):   
        print(id)
        handle = Entrez.esummary(db="sra", id=id)
        record = Entrez.read(handle)
        handle.close()
        #Handle terribly formatted XML
        Accessions=[]
        query_string=record[0]["Runs"]
        #don't blame me for python not having do while loops
        while True:
            extraction=extract(query_string,"Run acc=")
            if not extraction:
                break
            Accessions.append(extraction)
            query_string=query_string[query_string.find(extraction):]
        #Extract taxnomic info 
        try:
            taxid=int(extract(record[0]["ExpXml"].upper(),"TAXID="))
        except:
            taxid=""
        return [[i,taxid] for i in Accessions]

Entrez.email = snakemake.params[0]

db= "sra"
term = "(size+fractionation)&bacteria[Organism]&(sRNA|(small+RNA))"

#TODO in test if more querying is required
handle = Entrez.esearch(db = db, term = term, retmax= 100000)
record = Entrez.read(handle)
handle.close()

#TODO remove after testing
Accessions=[['SRR18745680', 208964],
['SRR18745681', 381754],
['SRR18745682', 446],
['SRR18745683', 1280],
['SRR18745684', 90371],
['SRR18745685', 511145],
['SRR18745686', 511145],
['SRR18745687', 511145],
['SRR18745688', 511145],
['SRR18745689', 511145],
['SRR18745690', 511145],
['SRR17278612', 469008],
['SRR17278613', 469008],
['SRR17730383', 83332],
['SRR17730384', 83332],
['SRR17730385', 83332],
['SRR17730386', 83332],
['SRR17730395', 83332],
['SRR17730396', 83332]]


Accessions=[] 
for id in record["IdList"]:
    Accessions+=getAccession(id)



In [10]:
#Output
with open(snakemake.output[0], "w",newline="") as f:
    writer = csv.writer(f,delimiter='\t', quoting=csv.QUOTE_NONE)
    writer.writerow(["Accession","TaxID"])
    writer.writerows(Accessions)